In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import warnings
import joblib
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1. 환경 설정
warnings.filterwarnings('ignore')
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# Python, Numpy, TensorFlow 등의 도구에 해한 재현성 확보
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 2. 경로 및 분석 타겟 설정
VGA_PATH = r'D:\minwoo\project\Virtual-Build-PC\last_data\VGA_Total'
TARGET_MODEL = "RTX 4060"

# 3. 데이터 로드 및 전처리
def load_vga_integrated(path, target):
    all_data = []
    files = sorted([f for f in os.listdir(path) if f.endswith('.csv')])
    for f in files:
        file_path = os.path.join(path, f)
        df_tmp = None
        for enc in ['utf-8', 'cp949']:
            try: df_tmp = pd.read_csv(file_path, encoding=enc); break
            except: continue
        if df_tmp is None: continue
        
        rows = df_tmp[df_tmp['Name'].str.contains(target, na=False, case=False)]
        cols = [c for c in df_tmp.columns if re.match(r'\d{4}-\d{2}-\d{2}', c)]
        for col in cols:
            p = pd.to_numeric(rows[col].astype(str).str.replace(',', '').str.extract('(\d+)')[0], errors='coerce')
            valid = p[p > 10000]
            if not valid.empty:
                all_data.append({'Date': col.split(' ')[0], 'Price': valid.mean()})
    
    df = pd.DataFrame(all_data).groupby('Date')['Price'].mean().reset_index()
    df['Date_dt'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date_dt')
    
    # 파생 변수 생성(Feature Engineering)
    df['Year'] = df['Date_dt'].dt.year  # 연도
    df['Month'] = df['Date_dt'].dt.month    # 월 (계절성 분석용)
    df['DayOfWeek'] = df['Date_dt'].dt.dayofweek    # 요일(주말/주중 패턴 분석)
    # AI 학습용 연속 시간 변수
    df['DaysFromStart'] = (df['Date_dt'] - df['Date_dt'].min()).dt.days
    
    # 노이즈 제거(Smoothing)
    df['Price_Raw'] = df['Price']
    df['Price_Smooth'] = df['Price'].rolling(window=3, min_periods=1).mean()
    return df

df = load_vga_integrated(VGA_PATH, TARGET_MODEL)

# 4. [EDA Overview] 주요 변수별 히스토그램 및 탐색적 분석
print(f"🔍 {TARGET_MODEL} 탐색적 데이터 분석(EDA) 리포트")
plt.figure(figsize=(20, 10))

# (1) 주요 변수 히스토그램 (가격 분포)
plt.subplot(2, 3, 1)
sns.histplot(df['Price_Raw'], kde=True, color='skyblue')
plt.title('주요 변수별 분포 (Price Histogram)')

# (2) 요일별 데이터 분포 히스토그램
plt.subplot(2, 3, 2)
sns.countplot(data=df, x='DayOfWeek', palette='viridis')
plt.title('요일별 데이터 분포')

# (3) 월별 데이터 분포 히스토그램
plt.subplot(2, 3, 3)
sns.countplot(data=df, x='Month', palette='magma')
plt.title('월별 데이터 분포')

# (4) 5x5 상관관계 히트맵 (Heatmap)
plt.subplot(2, 3, 4)
sns.heatmap(df[['Price','Year','Month','DayOfWeek','DaysFromStart']].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('변수 간 상관관계 분석')

# (5) 이상치 분석 (Boxplot)
plt.subplot(2, 3, 5)
sns.boxplot(x=df['Price_Raw'], color='lightcoral')
plt.title('시세 데이터 이상치(Outlier)')

# (6) 시세 추이 오버뷰
plt.subplot(2, 3, 6)
plt.plot(df['Date_dt'], df['Price_Raw'], alpha=0.5, label='Raw')
plt.plot(df['Date_dt'], df['Price_Smooth'], color='red', label='Trend')
plt.legend()
plt.title('시세 변동 Overview')

plt.tight_layout()
plt.show()

# 5. 모델 학습 데이터 준비
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['Price_Smooth']])
SEQ_LENGTH = 30
X, y = [], []
for i in range(len(scaled_data) - SEQ_LENGTH):
    X.append(scaled_data[i:i+SEQ_LENGTH])
    y.append(scaled_data[i+SEQ_LENGTH])
X, y = np.array(X), np.array(y)

# 6. LSTM 모델 빌드 및 학습
model = Sequential([
    # 입력층 : 30일치 데이터 1개 가격 받기
    Input(shape=(SEQ_LENGTH, 1)),   
    # 1 LSTM층 (128개 뉴런)
    LSTM(128, return_sequences=True),
    # 드롭아웃: 학습때 뉴런의 20%를 랜덤으로 끄기
    Dropout(0.2),
    # 두번째 LSTM층 (64개 뉴런)
    LSTM(64),
    # 출력층 (Dense) : 최종적으로 가격숫자 1개 찾기
    Dense(1)
])

# 내비게이션 알고리즘(Adam), 오차 계산 방식(huber)
model.compile(optimizer='adam', loss='huber')
# 같은 문제를 50번 반복, 한번에 문제 16개씩 묶어서 채점 및 학습 내용 수정, 문제 섞지 않기
model.fit(X, y, epochs=50, batch_size=16, verbose=1, shuffle=False)

# 7. 성능 평가지표 출력 (R2, MAE, MSE, RMSE)
y_pred_scaled = model.predict(X, verbose=0)
y_pred = scaler.inverse_transform(y_pred_scaled)    # 0 ~ 1 학습값을 (원)단위로 번역
y_actual = df['Price_Smooth'].values[SEQ_LENGTH:]   # 1 ~ 30 까지의 데이터 잘라낸 이후의 31부터 비교 및 채점

mse = mean_squared_error(y_actual, y_pred)
print("\n" + "="*30)
print("🤖 AI 모델 성능 평가 지표 (Overview)")
print("="*30)
print(f"1. R2 Score (결정계수): {r2_score(y_actual, y_pred):.4f}")
print(f"2. MAE (평균 절대 오차): {mean_absolute_error(y_actual, y_pred):,.0f}원")
print(f"3. MSE (평균 제곱 오차): {mse:,.0f}")
print(f"4. RMSE (평균 제곱근 오차): {np.sqrt(mse):,.0f}원")
print("="*30)

model.save("vga_model.h5")  # 학습 가중치 저장
joblib.dump(scaler, "vga_scaler.pkl")   # 0 ~ 1 변관 규칙 저장